In [94]:
from tqdm.notebook import tqdm
from random import seed, randint
from numpy import array, argmax
from math import ceil, log10, sqrt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, RepeatVector

In [2]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = [], []
    for i in range(n_examples):
        in_pattern = [randint(1, largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

In [6]:
X, y = random_sum_pairs(100, 4, 20)

In [8]:
print(X)

[[1, 9, 14, 5], [19, 2, 17, 9], [10, 11, 14, 12], [18, 4, 4, 15], [2, 13, 20, 11], [8, 7, 6, 1], [5, 7, 16, 4], [5, 19, 2, 17], [13, 6, 5, 5], [14, 1, 19, 17], [10, 5, 9, 3], [15, 4, 9, 2], [18, 17, 10, 7], [8, 12, 4, 18], [2, 20, 8, 19], [4, 11, 19, 3], [3, 4, 15, 16], [20, 14, 17, 13], [18, 20, 19, 15], [15, 14, 15, 18], [15, 7, 9, 20], [8, 17, 15, 10], [3, 13, 9, 8], [10, 15, 6, 8], [2, 10, 20, 1], [2, 5, 16, 19], [3, 10, 13, 2], [1, 4, 20, 14], [9, 6, 1, 7], [14, 5, 18, 6], [20, 20, 19, 6], [9, 19, 1, 19], [17, 13, 13, 7], [1, 11, 16, 19], [5, 1, 4, 7], [11, 12, 15, 6], [12, 8, 18, 20], [16, 11, 12, 12], [1, 14, 17, 14], [10, 9, 15, 9], [12, 5, 8, 8], [7, 7, 3, 3], [20, 2, 8, 11], [10, 18, 16, 1], [14, 12, 5, 13], [11, 12, 6, 16], [19, 14, 9, 17], [17, 9, 18, 20], [9, 10, 15, 3], [17, 6, 19, 17], [4, 7, 16, 7], [1, 12, 18, 8], [9, 2, 14, 20], [1, 19, 3, 14], [16, 5, 12, 15], [13, 7, 18, 11], [5, 19, 11, 17], [13, 9, 9, 18], [9, 1, 2, 2], [5, 6, 13, 17], [10, 1, 8, 12], [9, 8, 15, 7

In [9]:
print(y)

[29, 47, 47, 41, 46, 22, 32, 43, 29, 51, 27, 30, 52, 42, 49, 37, 38, 64, 72, 62, 51, 50, 33, 39, 33, 42, 28, 39, 23, 43, 65, 48, 50, 47, 17, 44, 58, 51, 46, 43, 33, 20, 41, 45, 44, 45, 59, 64, 37, 59, 34, 39, 45, 37, 48, 49, 52, 49, 14, 41, 31, 39, 32, 56, 31, 46, 65, 23, 21, 24, 52, 47, 35, 45, 45, 37, 59, 35, 51, 33, 48, 50, 43, 27, 35, 52, 46, 55, 58, 45, 42, 53, 27, 48, 58, 52, 37, 41, 37, 42]


In [14]:
def pairs_to_string(X, y, n_numbers, largest):
    max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
    Xstr = []
    for p in X:
        strp = '+'.join([str(n) for n in p])
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp
        Xstr.append(strp)
    max_length = ceil(log10(n_numbers * (largest+1)))
    ystr = []
    for p in y:
        strp = str(p)
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [17]:
pairs_to_string(X, y, 4, 20)

(['   1+9+14+5',
  '  19+2+17+9',
  '10+11+14+12',
  '  18+4+4+15',
  ' 2+13+20+11',
  '    8+7+6+1',
  '   5+7+16+4',
  '  5+19+2+17',
  '   13+6+5+5',
  ' 14+1+19+17',
  '   10+5+9+3',
  '   15+4+9+2',
  ' 18+17+10+7',
  '  8+12+4+18',
  '  2+20+8+19',
  '  4+11+19+3',
  '  3+4+15+16',
  '20+14+17+13',
  '18+20+19+15',
  '15+14+15+18',
  '  15+7+9+20',
  ' 8+17+15+10',
  '   3+13+9+8',
  '  10+15+6+8',
  '  2+10+20+1',
  '  2+5+16+19',
  '  3+10+13+2',
  '  1+4+20+14',
  '    9+6+1+7',
  '  14+5+18+6',
  ' 20+20+19+6',
  '  9+19+1+19',
  ' 17+13+13+7',
  ' 1+11+16+19',
  '    5+1+4+7',
  ' 11+12+15+6',
  ' 12+8+18+20',
  '16+11+12+12',
  ' 1+14+17+14',
  '  10+9+15+9',
  '   12+5+8+8',
  '    7+7+3+3',
  '  20+2+8+11',
  ' 10+18+16+1',
  ' 14+12+5+13',
  ' 11+12+6+16',
  ' 19+14+9+17',
  ' 17+9+18+20',
  '  9+10+15+3',
  ' 17+6+19+17',
  '   4+7+16+7',
  '  1+12+18+8',
  '  9+2+14+20',
  '  1+19+3+14',
  ' 16+5+12+15',
  ' 13+7+18+11',
  ' 5+19+11+17',
  '  13+9+9+18',
  '    9+1+2+2

In [18]:
Xstr, ystr = pairs_to_string(X, y, 4, 20)

In [74]:
def integer_encode(X, y, alphabets):
    char_to_int = dict((c, i) for i, c in enumerate(alphabets))
    Xenc = []
    for p in X:
        integer_encoded = [char_to_int[char] for char in p]
        Xenc.append(integer_encoded)
    yenc= []
    for p in y:
        integer_encoded = [char_to_int[char] for char in p]
        yenc.append(integer_encoded)
    return(Xenc, yenc)

In [43]:
integer_decode(Xstr, ystr, ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', ' ', '+'])

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}


([[10, 10, 10, 0, 11, 8, 11, 0, 3, 11, 4],
  [10, 10, 0, 8, 11, 1, 11, 0, 6, 11, 8],
  [0, 9, 11, 0, 0, 11, 0, 3, 11, 0, 1],
  [10, 10, 0, 7, 11, 3, 11, 3, 11, 0, 4],
  [10, 1, 11, 0, 2, 11, 1, 9, 11, 0, 0],
  [10, 10, 10, 10, 7, 11, 6, 11, 5, 11, 0],
  [10, 10, 10, 4, 11, 6, 11, 0, 5, 11, 3],
  [10, 10, 4, 11, 0, 8, 11, 1, 11, 0, 6],
  [10, 10, 10, 0, 2, 11, 5, 11, 4, 11, 4],
  [10, 0, 3, 11, 0, 11, 0, 8, 11, 0, 6],
  [10, 10, 10, 0, 9, 11, 4, 11, 8, 11, 2],
  [10, 10, 10, 0, 4, 11, 3, 11, 8, 11, 1],
  [10, 0, 7, 11, 0, 6, 11, 0, 9, 11, 6],
  [10, 10, 7, 11, 0, 1, 11, 3, 11, 0, 7],
  [10, 10, 1, 11, 1, 9, 11, 7, 11, 0, 8],
  [10, 10, 3, 11, 0, 0, 11, 0, 8, 11, 2],
  [10, 10, 2, 11, 3, 11, 0, 4, 11, 0, 5],
  [1, 9, 11, 0, 3, 11, 0, 6, 11, 0, 2],
  [0, 7, 11, 1, 9, 11, 0, 8, 11, 0, 4],
  [0, 4, 11, 0, 3, 11, 0, 4, 11, 0, 7],
  [10, 10, 0, 4, 11, 6, 11, 8, 11, 1, 9],
  [10, 7, 11, 0, 6, 11, 0, 4, 11, 0, 9],
  [10, 10, 10, 2, 11, 0, 2, 11, 8, 11, 7],
  [10, 10, 0, 9, 11, 0, 4, 11, 5, 11, 

In [59]:
def one_hot_encode(X, y, max_int):
    Xenc = []
    for p in X:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
    yenc = []
    for p in y:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [72]:
def generate_data(n_samples, n_numbers, largest, alphabets):
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    X, y = pairs_to_string(X, y, n_numbers, largest)
    X, y = integer_encode(X, y, alphabets)
    X, y = one_hot_encode(X, y, len(alphabets))
    X, y = array(X) , array(y)
    return X, y

In [61]:
def integer_decode(seq, alphabets):
    int_to_char = dict((i, c) for i, c in enumerate(alphabets))
    strings = []
    for p in seq:
        string = int_to_char[argmax(p)]
        strings.append(string)
    return ''.join(strings)

In [84]:
seed(1)
n_samples = 1000
n_numbers = 3
largest = 20
alphabets = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', ' ', '+']
n_char = len(alphabets)
n_in_seq_lenght = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
n_out_seq_lenght = ceil(log10(n_numbers * (largest+1)))

In [85]:
n_char

12

In [86]:
n_in_seq_lenght

8

In [87]:
n_out_seq_lenght

2

In [88]:
X, y = generate_data(n_samples, n_numbers, largest, alphabets)

In [89]:
X.shape

(1000, 8, 12)

In [90]:
X[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [91]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_in_seq_lenght, n_char)))
model.add(RepeatVector(n_out_seq_lenght))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_char, activation='softmax')))

In [96]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [93]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 repeat_vector (RepeatVector  (None, 2, 128)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 2, 64)             49408     
                                                                 
 time_distributed (TimeDistr  (None, 2, 12)            780       
 ibuted)                                                         
                                                                 
Total params: 122,380
Trainable params: 122,380
Non-trainable params: 0
_________________________________________________________________


In [97]:
for i in tqdm(range(200)):
    X, y = generate_data(n_samples, n_numbers, largest, alphabets)
    print('Epoch: ', i)
    model.fit(X, y, epochs=5, batch_size=50)

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch:  0
Epoch 1/5
20/20 [==============================] - 4s 9ms/step - loss: 2.4044 - accuracy: 0.2395
Epoch 2/5
20/20 [==============================] - 0s 11ms/step - loss: 2.1432 - accuracy: 0.2170
Epoch 3/5
20/20 [==============================] - 0s 10ms/step - loss: 1.9571 - accuracy: 0.2215
Epoch 4/5
20/20 [==============================] - 0s 9ms/step - loss: 1.9121 - accuracy: 0.2315
Epoch 5/5
20/20 [==============================] - 0s 9ms/step - loss: 1.8992 - accuracy: 0.2410
Epoch:  1
Epoch 1/5
20/20 [==============================] - 0s 11ms/step - loss: 1.8376 - accuracy: 0.2720
Epoch 2/5
20/20 [==============================] - 0s 9ms/step - loss: 1.7549 - accuracy: 0.3330
Epoch 3/5
20/20 [==============================] - 0s 9ms/step - loss: 1.6881 - accuracy: 0.3435
Epoch 4/5
20/20 [==============================] - 0s 9ms/step - loss: 1.6503 - accuracy: 0.3425
Epoch 5/5
20/20 [==============================] - 0s 9ms/step - loss: 1.6071 - accuracy: 0.3645
Epoch: 

20/20 [==============================] - 0s 9ms/step - loss: 0.7760 - accuracy: 0.7790
Epoch 4/5
20/20 [==============================] - 0s 8ms/step - loss: 0.7594 - accuracy: 0.7840
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.8240 - accuracy: 0.6740
Epoch:  17
Epoch 1/5
20/20 [==============================] - 0s 10ms/step - loss: 0.8121 - accuracy: 0.6880
Epoch 2/5
20/20 [==============================] - 0s 10ms/step - loss: 0.7614 - accuracy: 0.7540
Epoch 3/5
20/20 [==============================] - 0s 10ms/step - loss: 0.7180 - accuracy: 0.8175
Epoch 4/5
20/20 [==============================] - 0s 10ms/step - loss: 0.7018 - accuracy: 0.8290
Epoch 5/5
20/20 [==============================] - 0s 10ms/step - loss: 0.7006 - accuracy: 0.8075
Epoch:  18
Epoch 1/5
20/20 [==============================] - 0s 10ms/step - loss: 0.6970 - accuracy: 0.8155
Epoch 2/5
20/20 [==============================] - 0s 9ms/step - loss: 0.7126 - accuracy: 0.7625
Epoch 3/5
20

20/20 [==============================] - 0s 10ms/step - loss: 0.2451 - accuracy: 0.9810
Epoch:  33
Epoch 1/5
20/20 [==============================] - 0s 10ms/step - loss: 0.2524 - accuracy: 0.9765
Epoch 2/5
20/20 [==============================] - 0s 10ms/step - loss: 0.2582 - accuracy: 0.9685
Epoch 3/5
20/20 [==============================] - 0s 10ms/step - loss: 0.2527 - accuracy: 0.9770
Epoch 4/5
20/20 [==============================] - 0s 9ms/step - loss: 0.2365 - accuracy: 0.9825
Epoch 5/5
20/20 [==============================] - 0s 10ms/step - loss: 0.2264 - accuracy: 0.9845
Epoch:  34
Epoch 1/5
20/20 [==============================] - 0s 10ms/step - loss: 0.2359 - accuracy: 0.9805
Epoch 2/5
20/20 [==============================] - 0s 10ms/step - loss: 0.2143 - accuracy: 0.9870
Epoch 3/5
20/20 [==============================] - 0s 10ms/step - loss: 0.2207 - accuracy: 0.9880
Epoch 4/5
20/20 [==============================] - 0s 9ms/step - loss: 0.2260 - accuracy: 0.9790
Epoch 5/5


20/20 [==============================] - 0s 8ms/step - loss: 0.2790 - accuracy: 0.8880
Epoch 3/5
20/20 [==============================] - 0s 7ms/step - loss: 0.2254 - accuracy: 0.9135
Epoch 4/5
20/20 [==============================] - 0s 7ms/step - loss: 0.1281 - accuracy: 0.9810
Epoch 5/5
20/20 [==============================] - 0s 7ms/step - loss: 0.1063 - accuracy: 0.9930
Epoch:  50
Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.1018 - accuracy: 0.9905
Epoch 2/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0936 - accuracy: 0.9930
Epoch 3/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0876 - accuracy: 0.9950
Epoch 4/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0866 - accuracy: 0.9945
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0854 - accuracy: 0.9950
Epoch:  51
Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0918 - accuracy: 0.9935
Epoch 2/5
20/20 [=

...

20/20 [==============================] - 0s 8ms/step - loss: 0.0175 - accuracy: 0.9985
Epoch 3/5
20/20 [==============================] - 0s 7ms/step - loss: 0.0165 - accuracy: 0.9995
Epoch 4/5
20/20 [==============================] - 0s 7ms/step - loss: 0.0154 - accuracy: 0.9990
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0144 - accuracy: 0.9995
Epoch:  132
Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0188 - accuracy: 0.9975
Epoch 2/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0187 - accuracy: 0.9970
Epoch 3/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0168 - accuracy: 0.9985
Epoch 4/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0144 - accuracy: 0.9990
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0137 - accuracy: 0.9990
Epoch:  133
Epoch 1/5
20/20 [==============================] - 0s 9ms/step - loss: 0.0154 - accuracy: 0.9985
Epoch 2/5
20/20 

20/20 [==============================] - 0s 8ms/step - loss: 0.0113 - accuracy: 0.9995
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0102 - accuracy: 0.9995
Epoch:  148
Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0116 - accuracy: 0.9990
Epoch 2/5
20/20 [==============================] - 0s 9ms/step - loss: 0.0101 - accuracy: 1.0000
Epoch 3/5
20/20 [==============================] - 0s 9ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 4/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0095 - accuracy: 1.0000
Epoch:  149
Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0138 - accuracy: 0.9970
Epoch 2/5
20/20 [==============================] - 0s 9ms/step - loss: 0.0147 - accuracy: 0.9985
Epoch 3/5
20/20 [==============================] - 0s 9ms/step - loss: 0.0262 - accuracy: 0.9935
Epoch 4/5
20/20 

20/20 [==============================] - 0s 9ms/step - loss: 0.0095 - accuracy: 0.9990
Epoch 2/5
20/20 [==============================] - 0s 9ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 3/5
20/20 [==============================] - 0s 9ms/step - loss: 0.0082 - accuracy: 1.0000
Epoch 4/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0079 - accuracy: 1.0000
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch:  165
Epoch 1/5
20/20 [==============================] - 0s 11ms/step - loss: 0.0121 - accuracy: 0.9995
Epoch 2/5
20/20 [==============================] - 0s 9ms/step - loss: 0.0116 - accuracy: 0.9985
Epoch 3/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0108 - accuracy: 0.9995
Epoch 4/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0110 - accuracy: 1.0000
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0107 - accuracy: 0.9990
Epoch:  166
Epoch 1/5
20/20

20/20 [==============================] - 0s 8ms/step - loss: 0.0494 - accuracy: 0.9955
Epoch 4/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0286 - accuracy: 0.9980
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0239 - accuracy: 1.0000
Epoch:  181
Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0246 - accuracy: 0.9985
Epoch 2/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0213 - accuracy: 0.9985
Epoch 3/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0165 - accuracy: 0.9995
Epoch 4/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch:  182
Epoch 1/5
20/20 [==============================] - 0s 9ms/step - loss: 0.0175 - accuracy: 0.9970
Epoch 2/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0136 - accuracy: 1.0000
Epoch 3/5
20/20 

20/20 [==============================] - 0s 8ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch:  197
Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0088 - accuracy: 0.9980
Epoch 2/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0085 - accuracy: 0.9985
Epoch 3/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0075 - accuracy: 0.9985
Epoch 4/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0068 - accuracy: 0.9985
Epoch 5/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0062 - accuracy: 0.9995
Epoch:  198
Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0091 - accuracy: 0.9980
Epoch 2/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0074 - accuracy: 0.9985
Epoch 3/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0067 - accuracy: 0.9995
Epoch 4/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 5/5
20/20 

In [99]:
X, y = generate_data(n_samples, n_numbers, largest, alphabets)
result = model.predict(X, batch_size=50)

20/20 [==============================] - 1s 5ms/step


In [102]:
result[0]

array([[1.82349304e-05, 9.99981642e-01, 8.97717740e-08, 1.70536818e-09,
        1.91444495e-12, 3.29149109e-14, 3.42456785e-18, 2.80155576e-15,
        9.60558993e-11, 1.26021185e-08, 7.18314327e-16, 4.96190762e-11],
       [9.48503017e-01, 2.56474167e-02, 7.79769398e-05, 3.02791632e-05,
        3.90347488e-08, 5.64051965e-11, 5.04182196e-10, 3.91779423e-07,
        8.56947518e-05, 2.56551951e-02, 1.88073376e-11, 4.36697389e-09]],
      dtype=float32)

In [103]:
integer_decode(result[0], alphabets)

'21'

In [104]:
expected = [integer_decode(y1, alphabets) for y1 in y]
predicted = [integer_decode(y1, alphabets) for y1 in result]

In [108]:
list(zip(expected, predicted))

[('21', '21'),
 ('47', '46'),
 ('34', '34'),
 ('37', '37'),
 ('19', '19'),
 ('27', '27'),
 ('33', '33'),
 ('12', '13'),
 ('21', '21'),
 ('30', '30'),
 ('24', '24'),
 ('31', '31'),
 ('32', '32'),
 ('19', '19'),
 ('38', '38'),
 ('39', '39'),
 ('40', '40'),
 ('18', '18'),
 ('10', '10'),
 ('36', '36'),
 ('38', '38'),
 ('35', '35'),
 ('17', '18'),
 ('20', '20'),
 ('33', '33'),
 ('25', '25'),
 ('32', '32'),
 ('30', '30'),
 ('25', '25'),
 ('30', '30'),
 ('34', '34'),
 ('27', '27'),
 ('39', '39'),
 ('19', '19'),
 ('27', '27'),
 (' 9', ' 9'),
 ('30', '30'),
 ('51', '50'),
 ('27', '27'),
 ('26', '26'),
 ('27', '27'),
 ('22', '22'),
 ('24', '24'),
 ('34', '34'),
 ('31', '31'),
 ('39', '39'),
 ('28', '28'),
 ('43', '43'),
 ('13', '14'),
 ('17', '18'),
 ('24', '24'),
 ('43', '43'),
 ('28', '28'),
 ('33', '33'),
 ('20', '20'),
 ('34', '34'),
 ('37', '37'),
 ('32', '32'),
 ('27', '27'),
 ('47', '47'),
 ('43', '42'),
 ('36', '36'),
 ('48', '48'),
 ('38', '38'),
 ('32', '33'),
 ('31', '31'),
 ('26', '2

In [116]:
abs(array(expected, dtype='int') - array(predicted, dtype='int')).sum() / len(expected) * 100

10.6